## Decision tree

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [2]:
#Menggunakan pandas untuk membaca file CSV dari URL dan menyimpannya ke dalam Dataframe df
df = pd.read_csv("https://raw.githubusercontent.com/rahmantaufik27/dataset/main/study/daily_weather_prediction.csv")

In [ ]:
#Menampilkan seluruh isi DataFrame df
df

## EDA (Exploratory Data Analysis)

In [4]:
#Menampilkan informasi tentang DataFrame df, termasuk jumlah baris dan kolom, tipe data setiap kolom
#serta jumlah nilai non-null dan pengunaan memori
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   number                  1095 non-null   int64  
 1   air_pressure_9am        1092 non-null   float64
 2   air_temp_9am            1090 non-null   float64
 3   avg_wind_direction_9am  1091 non-null   float64
 4   avg_wind_speed_9am      1092 non-null   float64
 5   max_wind_direction_9am  1092 non-null   float64
 6   max_wind_speed_9am      1091 non-null   float64
 7   rain_accumulation_9am   1089 non-null   float64
 8   rain_duration_9am       1092 non-null   float64
 9   relative_humidity_9am   1095 non-null   float64
 10  relative_humidity_3pm   1095 non-null   float64
dtypes: float64(10), int64(1)
memory usage: 94.2 KB


In [5]:
df.isnull().sum()

number                    0
air_pressure_9am          3
air_temp_9am              5
avg_wind_direction_9am    4
avg_wind_speed_9am        3
max_wind_direction_9am    3
max_wind_speed_9am        4
rain_accumulation_9am     6
rain_duration_9am         3
relative_humidity_9am     0
relative_humidity_3pm     0
dtype: int64

In [6]:
#Menggunakan method describe() untuk mendapatkan ringkasan statistik dari data numerik dalam DataFrame df
df.describe()

,number,air_pressure_9am,air_temp_9am,avg_wind_direction_9am,avg_wind_speed_9am,max_wind_direction_9am,max_wind_speed_9am,rain_accumulation_9am,rain_duration_9am,relative_humidity_9am,relative_humidity_3pm
count,1095.000000,1092.000000,1090.000000,1091.000000,1092.000000,1092.000000,1091.000000,1089.000000,1092.000000,1095.000000,1095.000000
mean,547.000000,918.882551,64.933001,142.235511,5.508284,148.953518,7.019514,0.203079,294.108052,34.241402,35.344727
std,316.243577,3.184161,11.175514,69.137859,4.552813,67.238013,5.598209,1.593952,1598.078779,25.472067,22.524079
min,0.000000,907.990000,36.752000,15.500000,0.693451,28.900000,1.185578,0.000000,0.000000,6.090000,5.300000
25%,273.500000,916.550000,57.281000,65.972506,2.248768,76.553003,3.067477,0.000000,0.000000,15.092243,17.395000
50%,547.000000,918.921045,65.715479,166.000000,3.871333,177.300000,4.943637,0.000000,0.000000,23.179259,24.380000
75%,820.500000,921.160073,73.450974,191.000000,7.337163,201.233153,8.947760,0.000000,0.000000,45.400000,52.060000
max,1094.000000,929.320000,98.906000,343.400000,23.554978,312.200000,29.840780,24.020000,17704.000000,92.620000,92.250000


## Preprocessing

In [7]:
#Menggunakan method dropna() untuk menghapus baris yang mengandung nilai null dari dataframe df
#dan menyimpan dataframe yang telah dimodifikasi kembali ke variabel df
df = df.dropna()

In [8]:
#Menambahkan kolom baru 'high_humidity_label' ke dataframe df
df['high_humidity_label'] = (df['relative_humidity_3pm'] > 24.99) * 1

In [9]:
#Mendapatkan list nama-nama kolom dari DataFrame df
features = list(df.columns)

#Menghapus kolom 'number' dari list fitur
features.remove('number')

#Menghapus kolom 'relative_humidity_9am' dari list fitur
features.remove('relative_humidity_9am')

#Menghapus kolom 'relative_humidity_3pm' dari list fitur
features.remove('relative_humidity_3pm')

#Menghapus kolom 'high_humidity_label' dari list fitur
features.remove('high_humidity_label')

#Menampilkan list fitur yang telah dimodifikasi
features

['air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am']

In [10]:
#Memilih subset dari DataFrame df yang hanya terdiri dari fitur-fitur yang telah dipersiapkan sebelumnya
x = df[features].copy()

#Memilih kolom 'high_humidity_label' dari DataFrame df sebagai target variabel yang akan diprediksi
y = df["high_humidity_label"].copy()

In [11]:
train_pct_index = int(0.8 * len(x))
x_train, x_test = x[:train_pct_index], x[train_pct_index:]
y_train, y_test = y[:train_pct_index], y[train_pct_index:]

## Modelling

In [12]:
#Membuat objek DecisionTreeClassifier dengan mengatur jumlah maksimum leaf nodes (simpul daun) menjadi 10
#dan menetapkan nilai random_state agar hasilnya dapat direproduksi dengan konsisten
dt_cls = DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

#Melatih model DecisionTreeClassifier menggunakan data pelatihan yang telah dipersiapkan
#Ini dilakukan dengan menggunakan metode fit(), di mana x_train adalah fitur-fitur pelatihan
#dan y_train adalah label pelatihan yang ingin diprediksi oleh model
dt_cls.fit(x_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

## Prediction

In [13]:
#Melakukan prediksi menggunakan model DecisionTreeClassifier yang telah dilatih sebelumnya
#Pada data pengujian x_test, ini dilakukan dengan menggunakan metode predict()
y_pred = dt_cls.predict(x_test)

#Menampilkan hasil prediksi yang disimpan dalam variabel y_pred
y_pred

array([1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [14]:
#Menghitung akurasi antara label yang sebenarnya (y_test) dan label yang diprediksi (y_pred)
acc = accuracy_score(y_test, y_pred)

#Menampilkan akurasi dalam bentuk persentase dengan dua angka desimal
print("Akurasi: %.2f%%" % (acc * 100))

Akurasi: 78.87%


## Conclusion
Dengan menggunakan model Decision Tree Classifier dengan batasan maksimum 10 leaf nodes, akurasi prediksi pada data pengujian adalah sekitar 78.87% Ini menunjukkan bahwa model tersebut dapat dengan tepat memprediksi label "High_Humidity" dengan tingkat keberhasilan sekitar 78.87% berdasarkan fitur-fitur yang telah dipilih.